# Get Book IDs for Fantasy Books

In [1]:
#!pip install goodreads-api-client
#!pip install xmltodict
#import goodreads_api_client as gr
#from goodreads import client
import requests
import pandas as pd
import numpy as np
import pickle

In [2]:
from bs4 import BeautifulSoup
import xmltodict
import pprint
import json
import os
#!pip install selenium
#from selenium import webdriver
#from selenium.webdriver.common.keys import Keys
#chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
#os.environ["webdriver.chrome.driver"] = chromedriver

**Define helpful functions**

In [3]:
def get_info(url):
    key = api_key
    response = requests.get(url, params={'key': api_key})
    #soup = BeautifulSoup(response.content, "lxml-xml")
    results = xmltodict.parse(response.content)
    #results = json.dumps(results)
    return(results['GoodreadsResponse'])

In [4]:
def get_soup(url):
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    #print(soup.prettify())
    return(soup)

def get_soup_selenium(url):
    driver = webdriver.Chrome(chromedriver)
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    return(soup)
def stripHTML(v):
    v = [re.sub(r'\t', '', x) for x in v]
    v = [re.sub(r'\r', '', x) for x in v]
    v = [re.sub(r'\n', '', x) for x in v]
    return(v)

In [5]:
def get_count(soup):
    count = soup.find_all('div',
                          {'class': 'inter loading uitext'})[1].text.split('of ')[1].split(' loaded')[0]


    
def get_maxpage(soup):
    page_links = []
    for link in soup.find_all('a'):
            #links.append(link['href'])
        try:
            if '?page=' in str(link['href']):
                page_links.append(str(link['href']))
        except:
            pass

    max_page = page_links[-2].split('?page=')[1]
    return(max_page)
    #print(max_page, count)

In [6]:
userids = pd.read_pickle('user_ids.pkl')
len(userids)

449856

In [7]:
ratings_all_users = pd.read_pickle('user_book_ratings_scraped.pkl')
len(ratings_all_users)

6730

In [8]:
print('The last user scraped is:', ratings_all_users[-1]['userid'], 
      'This is last index of user_ids:', userids.index(ratings_all_users[-1]['userid']))

The last user scraped is: 11454689 This is last index of user_ids: 10997


In [9]:
iters = userids.index(ratings_all_users[-1]['userid'])
iters

10997

# Building users' ratings dict with web scraping. Starting with user 1000.

In [ ]:
#ratings_all_users = [] # [{userid: gid, ratings: [{bookid: bookid, rating: stars}]}]
#for gid in userids[0:1]:
for count, gid in enumerate(userids[iters+1:]):  
    ratings_user_i = {} # user_ratings_dict = {'userid':gid, 'ratings': user_books_rating_list}
    ratings_user_i['userid'] = gid
    
    try:
        soup1 = get_soup('https://www.goodreads.com/review/list/'+gid)
        max_page = get_maxpage(soup1)  

        user_ratings_all = []

        for page in range(1, int(max_page)):

            url = 'https://www.goodreads.com/review/list/'+gid+'?page='+str(page)
            soup = get_soup(url)

            tables = soup.find_all("table")
            #tables[1]
            rows = tables[1].find_all('tr')

            tds = [row.find_all('td') for row in rows]

            for i, td in enumerate(tds):
                user_ratings = {}
                try:
                    user_ratings['rating'] = tds[i][13].find('span', {'class': 'staticStars'}).text
                    user_ratings['bookid'] = tds[i][2].find('div', 
                                    {'class': "js-tooltipTrigger tooltipTrigger"})['data-resource-id']
                    user_ratings_all.append(user_ratings)
                except:
                    pass

        ratings_user_i['ratings'] = user_ratings_all

        ratings_all_users.append(ratings_user_i)

        if count % 50 == 0:
            with open('user_book_ratings_scraped.pkl', 'wb') as f:
                pickle.dump(ratings_all_users, f)
        print(count+iters+1)
    except Exception as e:
        print(count+iters+1, e)
        #pass
  

10998 list index out of range
10999
11000 list index out of range
11001
11002 list index out of range
11003 list index out of range
11004
11005
11006 list index out of range
11007 list index out of range
11008 list index out of range
11009
11010
11011 list index out of range
11012 list index out of range
11013
11014 list index out of range
11015


In [ ]:
len(ratings_all_users)

In [ ]:
with open('user_book_ratings_scraped.pkl', 'wb') as f:
    pickle.dump(ratings_all_users, f)

**END NOTEBOOK**